# Experiment 1
### Cedric Chauve, 11/12/2018

## Introduction

In this experiment (script *exp1a.sh*) we counted the number of histories for the following data:
- species tree size (number of leaves) from 3 to 32 (exp1a) and 33 to 64 (exp1b),
- for each species tree size, we considered 100 trees 
    - the first one (index 0) is the caterpillar,
    - if k is a power of 2 the second tree (index 1) is the complete binary tree,
    - the remaining trees are random,
- the history size (number of leaves) ranges from 1 to 50 (exp1a) and 1 to 128 (exp1b).

We record the results for species trees of a given size *k* in the file *results/exp1a_k*. Each non-comment row of the result file has the following tab-separated format:
- species tree size,
- species tree index,
- ranking type (U for unranked, we do not consider ranked trees),
- newick string describing the tree,
- number of histories separated by spaces.

For each configuration, we count the number of histories in two models, one with only DL histories and one with DLT histories.

In [5]:
import csv
import pandas as pd
import numpy as np
import gzip
import io

In [17]:
# Parameters

# Number of species trees
NB_S_TREES    = 100
S_TREES_INDEX = [i for i in range(0,NB_S_TREES)]
# Evolutionary models
EVOL_MODELS = [('U','DL'),('U','DLT')]

In [65]:
# Format: RESULTS[evol_model][s][n][tree_index] is 
# the number of histories of size n for tree tree_index of size s in model evol_model

def read_results(S_SIZES,H_SIZES,S_TREES_INDEX,PREFIX):
    RESULTS = {x:{s:{n:{t:0 for t in S_TREES_INDEX} for n in H_SIZES}  for s in S_SIZES} for x in EVOL_MODELS}
    for s in S_SIZES:
        with gzip.open('../results/'+PREFIX+'_'+str(s)+'.gz', 'r') as f:
            reader = csv.reader(io.TextIOWrapper(f, newline=""),delimiter='\t')
            for row in reader:
                if row[0][0]!='#':
                    model = (row[2],row[3])
                    t_ind = int(row[1])
                    row5  = row[5].split()
                    for n in H_SIZES:
                        RESULTS[model][s][n][t_ind] = int(row5[n-1])
                    
    RESULTS_frame = pd.DataFrame.from_dict({(m,s,n): RESULTS[m][s][n] 
                                            for m in RESULTS.keys() 
                                            for s in RESULTS[m].keys()
                                            for n in RESULTS[m][s].keys()},
                                            orient='index')
    return((RESULTS,RESULTS_frame))

def compute_stats(S_SIZES,H_SIZES,S_TREES_INDEX):
    STATS = {x:{s:{n:{} for n in H_SIZES}  for s in S_SIZES} for x in EVOL_MODELS}

    for x in EVOL_MODELS:
        for s in S_SIZES:
            for n in H_SIZES:
                data =  np.array([RESULTS[x][s][n][t] for t in S_TREES_INDEX])
                STATS[x][s][n] = {'avg':np.mean(data), 'std':np.std(data), 'max/min':np.max(data)/np.min(data)}
            
    STATS_frame = pd.DataFrame.from_dict({(m,s,n): STATS[m][s][n] 
                                         for m in STATS.keys() 
                                         for s in STATS[m].keys()
                                         for n in STATS[m][s].keys()},
                                         orient='index')
    return((STATS,STATS_frame))

def compute_ratio_DL_DLT(RESULTS,S_SIZES,H_SIZES,S_TREES_INDEX):
    RATIOS = {s:{n:{} for n in H_SIZES}  for s in S_SIZES}
    for s in S_SIZES:
        for n in H_SIZES:
            ratios = np.array([RESULTS[('U','DLT')][s][n][t]/RESULTS[('U','DL')][s][n][t] for t in S_TREES_INDEX])
            RATIOS[s][n] = {'avg':np.mean(ratios), 'std':np.std(ratios), 'min':np.min(ratios), 'max':np.max(ratios), 'max/min':np.max(ratios)/np.min(ratios)}
            
    RATIOS_frame = pd.DataFrame.from_dict({(s,n): RATIOS[s][n]
                                          for s in RATIOS.keys()
                                          for n in RATIOS[s].keys()},
                                          orient='index')
    return((RATIOS,RATIOS_frame))

## Experiment exp1a

### Analysis 1.
The first analysis just look at the number of histories for each pair *(s,n)* (*s* = species tree size, *n* = histories size). For each selected pair, we look at the average number of histories, the standard deviation and the ration *max/min*.

In [36]:
# Analyse 1: average, standard deviation, ratio min and max for the number of histories per model for a given species tree size
S_SIZES_1a = [4,8,16,32]
H_SIZES_1a = [10,20,30,40,50]

(RESULTS_1a,RESULTS_1a_frame) = read_results(S_SIZES_1a,H_SIZES_1a,S_TREES_INDEX,'exp1a')
(STATS_1a_1,STATS_1a_1_frame) = compute_stats(S_SIZES_1a,H_SIZES_1a, S_TREES_INDEX)

In [58]:
np.std([RESULTS_1a[('U','DLT')][4][10][t]/RESULTS_1a[('U','DL')][4][10][t] for t in S_TREES_INDEX])

28.96211710207647

In [37]:
STATS_1a_1_frame

avg            std       max/min
(U, DL)  4  10   2.068155e+11   4.618451e+10  2.738565e+00
            20   2.402022e+24   7.350661e+23  6.497603e+00
            30   4.578739e+37   1.569684e+37  1.572603e+01
            40   1.059871e+51   3.797903e+50  3.822820e+01
            50   2.720679e+64   9.924259e+63  9.305067e+01
         8  10   1.910970e+15   1.566116e+15  3.566652e+01
            20   2.474365e+32   2.986438e+32  8.444849e+02
            30   6.030736e+49   8.544215e+49  2.028594e+04
            40   1.910940e+67   2.946170e+67  4.958215e+05
            50   6.934493e+84   1.125588e+85  1.220343e+07
         16 10   3.131188e+19   4.936552e+19  1.778540e+03
            20   1.158103e+41   3.311438e+41  1.786056e+06
            30   9.392178e+62   3.738479e+63  1.655011e+09
            40   1.096907e+85   5.499493e+85  1.546261e+12
            50  1.574578e+107  9.412679e+107  1.461798e+15
         32 10   8.074475e+23   4.020622e+24  2.350184e+05
            20   6.037014e+50   5.708133e+51  3.389165e+10
            30   1.065871e+78   1.056097e+79  3.950302e+15
            40  2.265465e+105  2.253211e+106  4.420411e+20
            50  5.303012e+132  5.276213e+133  4.915373e+25
(U, DLT) 4  10   8.361351e+11   2.562197e+12  2.893654e+02
            20   3.197647e+26   2.525534e+27  6.044444e+04
            30   4.049413e+41   3.659581e+42  1.116644e+07
            40   6.584081e+56   6.247799e+57  1.998768e+09
            50   1.204631e+72   1.170706e+73  3.546274e+11
         8  10   5.581653e+16   3.864497e+17  2.206967e+04
            20   1.159846e+37   1.099247e+38  6.476896e+08
            30   4.603812e+57   4.535898e+58  1.686406e+13
            40   2.182951e+78   2.167807e+79  4.228212e+17
            50   1.129880e+99  1.123792e+100  1.041010e+22
         16 10   4.703975e+21   3.359650e+22  1.090762e+06
            20   1.470521e+47   1.283540e+48  2.815114e+12
            30   8.071030e+72   7.619346e+73  6.617901e+18
            40   5.489541e+98   5.353241e+99  1.518924e+25
            50  4.170162e+124  4.117914e+125  3.451349e+31
         32 10   4.432911e+26   3.311644e+27  2.320288e+07
            20   2.280848e+57   1.956197e+58  2.291334e+15
            30   2.076097e+88   1.937041e+89  2.203206e+23
            40  2.415580e+119  2.343248e+120  2.121629e+31
            50  3.190477e+150  3.143983e+151  2.045502e+39

### Comments.
For both the DL and DLT models, the standard deviation is larger than the mean, indicating a very large spread of the distribution of the number of histories. This is also illustrated by the very large ration *max/min*.

### Analysis 2. 
We look at the ratio between the number of DLT-histories and the number of DL-histories.

In [66]:
(RATIOS_DLT_DL_1a,RATIOS_DLT_DL_1a_frame) = compute_ratio_DL_DLT(RESULTS_1a,S_SIZES_1a,H_SIZES_1a,S_TREES_INDEX)

In [67]:
RATIOS_DLT_DL_1a_frame

avg           std        min           max       max/min
4  10  5.651160e+00  2.896212e+01   1.000000  2.893654e+02  2.893654e+02
   20  6.311465e+02  6.014839e+03   1.000000  6.044444e+04  6.044444e+04
   30  1.124141e+05  1.110995e+06   1.000000  1.116644e+07  1.116644e+07
   40  2.001345e+07  1.988725e+08   1.000000  1.998768e+09  1.998768e+09
   50  3.547184e+09  3.528489e+10   1.000000  3.546274e+11  3.546274e+11
8  10  3.144372e+02  3.062889e+03   1.000000  3.078902e+04  3.078902e+04
   20  1.222064e+07  1.215862e+08   1.000000  1.221988e+09  1.221988e+09
   30  4.319341e+11  4.297687e+12   1.000000  4.319338e+13  4.319338e+13
   40  1.471052e+16  1.463678e+17   1.000000  1.471052e+18  1.471052e+18
   50  4.920755e+20  4.896090e+21   1.000000  4.920755e+22  4.920755e+22
16 10  2.058410e+04  2.046169e+05   1.087339  2.056497e+06  1.891312e+06
   20  9.431860e+10  9.384582e+11   1.173507  9.431859e+12  8.037325e+12
   30  3.969785e+17  3.949886e+18   1.267679  3.969785e+19  3.131537e+19
   40  1.633196e+24  1.625009e+25   1.369610  1.633196e+26  1.192453e+26
   50  6.654569e+30  6.621212e+31   1.479811  6.654569e+32  4.496906e+32
32 10  7.650579e+05  7.611873e+06   1.651048  7.650224e+07  4.633557e+07
   20  2.014785e+14  2.004686e+15   2.615439  2.014785e+16  7.703429e+15
   30  4.930014e+22  4.905302e+23   4.128517  4.930014e+24  1.194137e+24
   40  1.184790e+31  1.178851e+32   6.526182  1.184790e+33  1.815442e+32
   50  2.826641e+39  2.812472e+40  10.327198  2.826641e+41  2.737084e+40

### Comments.
Again, a very large spread, as well as a quick increase of the ratio. This goes along the intuition that the search space grows very quickly when transfers are added to the model. 

## Experiment exp1b

The numbers of histories are too large for python:
"OverflowError: cannot convert float infinity to integer"
